# **Automated Research Paper Analysis Tool**

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import nltk

# Download the required 'punkt' tokenizer data
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk

# Specify a directory to store nltk data
nltk.data.path.append('/usr/nltk_data')

# Download the 'punkt' tokenizer
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=da379bccdc9331b233f7b9d222de5794668e4f242d503c693526788877db8733
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install rouge_score


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00


**Import necessary libraries**

In [ ]:
# Import necessary libraries
from transformers import pipeline
import nltk
from nltk.translate.bleu_score import sentence_bleu
from evaluate import load

# Download NLTK resources (Ensure 'punkt' and any other required resources are available)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Define a simple research paper text (example)**

In [ ]:
# Define a simple research paper text (example)
research_paper_text = """
AI in Cybersecurity – Enhancing Threat Detection and Response
Introduction
As digital technologies advance, cybersecurity threats have grown in complexity, creating significant challenges for individuals, organizations, and governments.
Cybercriminals now exploit sophisticated techniques, making traditional security solutions inadequate. Artificial Intelligence (AI) offers new avenues for addressing these
challenges by enabling advanced threat detection, predictive analytics, and automated incident response.
AI’s Role in Threat Detection
AI has redefined threat detection by surpassing the limitations of traditional rule-based security systems. Conventional systems rely heavily on signature-based methods,
which are ineffective against zero-day attacks and new, unknown malware. AI, through machine learning (ML) and deep learning (DL), learns from historical data and detects anomalies,
even in complex attack scenarios.
AI-Driven Incident Response
AI significantly enhances incident response capabilities, enabling security teams to act quickly and effectively. Automated response systems powered by AI can isolate affected systems,
neutralize threats, and restore operations with minimal human intervention.AI facilitates real-time threat intelligence by processing global cybersecurity data from diverse sources, such as threat feeds, darknet forums, and malware analysis repositories. This information is distilled into actionable insights that inform security teams of emerging attack trends, newly discovered vulnerabilities, and potential adversary tactics.
By leveraging this intelligence, organizations can adjust their defenses dynamically, staying ahead of cybercriminals.The future of AI in cybersecurity lies in the convergence of advanced technologies such as blockchain, quantum computing, and edge computing. Blockchain can enhance data integrity and transparency, while quantum-resistant algorithms will prepare defenses against future quantum-based attacks. Edge AI, deployed closer to data sources, enables real-time threat detection and response in distributed environments, such as IoT networks.

Additionally, the emergence of explainable AI (XAI) is set to address the "black-box" nature of current models, providing transparency into
 AI decision-making processes. This not only improves trust but also ensures regulatory compliance.
"""

**function to analyze the research paper (generate summary and questions)**

In [ ]:
# Define preprocessing function (simple sentence tokenization)
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)  # Tokenize into sentences
    return ' '.join(sentences)  # Return as a single string for summarization

# Define the function to analyze the research paper (generate summary and questions)
def analyze_research_paper(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    max_length = min(130, len(text.split()))  # Prevent overly long summaries
    summary = summarizer(
        text,
        max_length=max_length,
        min_length=30,  # Ensure the summary isn't too short
        do_sample=False
    )[0]['summary_text']

    question_generator = pipeline("text2text-generation", model="valhalla/t5-small-qg-prepend")
    questions = question_generator(
        "generate questions: " + text,
        max_length=100,
        num_return_sequences=3,
        num_beams=5
    )

    generated_questions = [q['generated_text'] for q in questions]
    return summary, generated_questions



**model evaluation**

In [ ]:
# Define model evaluation function (ROUGE for Summarization, BLEU for Question Generation)
def evaluate_model(generated_summary, reference_summary, generated_questions, reference_questions):
    # ROUGE Evaluation for Summarization
    rouge_metric = load("rouge")
    rouge_results = rouge_metric.compute(predictions=[generated_summary], references=[reference_summary])
    print(f"ROUGE Score: {rouge_results}")

    # BLEU Evaluation for Question Generation
    bleu_scores = []
    for generated, reference in zip(generated_questions, reference_questions):
        bleu_score = sentence_bleu([reference.split()], generated.split())
        bleu_scores.append(bleu_score)
        print(f"BLEU Score for generated question: '{generated}' and reference: '{reference}' = {bleu_score:.4f}")

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0
    print(f"Average BLEU Score: {avg_bleu_score:.4f}")

# Preprocess the research paper text
preprocessed_text = preprocess_text(research_paper_text)

# Analyze the text (generate summary and questions)
generated_summary, generated_questions = analyze_research_paper(preprocessed_text)

# Define reference summary and questions for evaluation
reference_summary = "AI enhances cybersecurity by improving threat detection and enabling automated incident response."
reference_questions = [
    "How does AI help improve cybersecurity?",
    "What is AI's role in threat detection?",
    "How does AI contribute to incident response?"
]

# Print generated summary and questions
print("\033[1m📘 MAIN OUTPUT\033[0m")
print("\033[1m📄 Generated Summary:\033[0m")
print(generated_summary)
print("\n\033[1m❓ Generated Questions:\033[0m")
for question in generated_questions:
    print("-", question)


print("\n\033[1m📊 EVALUATION METRICS\033[0m")
print("\033[1m🔍 ROUGE Scores (Summarization):\033[0m")
# Evaluate the model using ROUGE and BLEU
evaluate_model(generated_summary, reference_summary, generated_questions, reference_questions)


📘 MAIN OUTPUT
📄 Generated Summary:
Artificial Intelligence (AI) enables advanced threat detection, predictive analytics, and automated incident response. The future of AI in cybersecurity lies in the convergence of advanced technologies such as blockchain, quantum computing, and edge computing.

❓ Generated Questions:
- What is AI's role in Threat Detection and Response?
- What is AI's role in Threat Detection?
- AI in Cybersecurity – Enhancing Threat Detection and Response Introduction

📊 EVALUATION METRICS
🔍 ROUGE Scores (Summarization):
ROUGE Score: {'rouge1': 0.3478260869565218, 'rouge2': 0.13636363636363635, 'rougeL': 0.3043478260869565, 'rougeLsum': 0.3043478260869565}
BLEU Score for generated question: 'What is AI's role in Threat Detection and Response?' and reference: 'How does AI help improve cybersecurity?' = 0.0000
BLEU Score for generated question: 'What is AI's role in Threat Detection?' and reference: 'What is AI's role in threat detection?' = 0.6148
BLEU Score for gener